# NewBucket Implementation Analysis

## Token Bucket Rate Limiter with Detailed Commentary

This notebook provides a comprehensive analysis of the `NewBucket` class implementation, including:
- Fully commented source code
- Architectural design decisions
- Usage examples and demonstrations
- Potential improvements and best practices

---
## Part 1: Core Implementation with Comments

In [ ]:
from datetime import datetime, timedelta

class NewBucket:
    """
    Token Bucket Rate Limiter Implementation
    
    This class implements a token bucket algorithm for rate limiting,
    where tokens (quota) are consumed over time and can be refilled
    periodically.
    """
    
    def __init__(self, period):
        """
        Initialize the bucket with a time period.
        
        Args:
            period: Time window in seconds for the bucket's rate limiting period
        """
        # Convert period to timedelta for proper time arithmetic
        self.period_delta = timedelta(seconds=period)
        
        # Track when the bucket was last reset/created
        self.reset_time = datetime.now()
        
        # Maximum capacity of the bucket (total tokens available)
        self.max_quota = 0
        
        # Tokens that have been used/consumed from the bucket
        self.quota_consumed = 0

    def __repr__(self):
        """
        String representation for debugging and logging.
        Shows current state of quota consumption.
        """
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')

    @property
    def quota(self):
        """
        Property getter: Calculate available quota.
        
        Returns:
            int: Available tokens (max_quota - consumed)
        
        Design Decision: Using a calculated property rather than storing
        available quota separately ensures consistency and eliminates
        potential synchronization issues.
        """
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        """
        Property setter: Handle quota assignment with three distinct scenarios.
        
        Args:
            amount: The new quota value to set
        
        This setter implements intelligent state transitions rather than
        direct assignment, maintaining internal consistency through
        calculated adjustments.
        """
        # Calculate the difference between current max and requested amount
        delta = self.max_quota - amount
        
        # SCENARIO 1: Reset/Empty the bucket
        if amount == 0:
            # Complete reset - both capacity and consumption go to zero
            self.quota_consumed = 0
            self.max_quota = 0
            
        # SCENARIO 2: Initial fill or capacity increase
        elif delta < 0:
            # Delta < 0 means amount > max_quota (requesting more than current capacity)
            # Assertion ensures we're starting fresh (no partial consumption)
            assert self.quota_consumed == 0, "Cannot increase capacity with consumed quota"
            # Set new maximum capacity
            self.max_quota = amount
            
        # SCENARIO 3: Consuming quota (normal usage)
        else:
            # Delta >= 0 means amount <= max_quota (consuming from available quota)
            # Assertion validates bucket integrity (consumed shouldn't exceed max)
            assert self.max_quota >= self.quota_consumed, "Quota consumed exceeds maximum"
            # Increase consumption by the delta amount
            self.quota_consumed += delta

---
## Part 2: Helper Function

In [ ]:
def fill(bucket, amount):
    """
    Fill the bucket with tokens.
    
    This function adds tokens to the bucket by setting its quota.
    When filling an empty bucket, this triggers the capacity setting logic.
    
    Args:
        bucket: NewBucket instance to fill
        amount: Number of tokens to add
    """
    bucket.quota = amount

---
## Part 3: Basic Usage Demonstration

In [ ]:
# Create a bucket with 60-second period
bucket = NewBucket(60)
print('Initial:', bucket)
print('Available quota:', bucket.quota)
print()

In [ ]:
# Fill the bucket with 100 tokens
fill(bucket, 100)
print('Filled:', bucket)
print('Available quota:', bucket.quota)
print()

---
## Part 4: Demonstrating Token Consumption

In [ ]:
# Consume some tokens by setting quota to a lower value
print(f"Before consumption: {bucket.quota} tokens available")

# Consume 30 tokens (setting quota to 70)
bucket.quota = 70
print(f"After consuming 30 tokens: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

In [ ]:
# Consume more tokens
bucket.quota = 20
print(f"After consuming 50 more tokens: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

---
## Part 5: Demonstrating Reset

In [ ]:
# Reset the bucket
bucket.quota = 0
print(f"After reset: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

---
## Part 6: Design Analysis

### Key Design Decisions

#### 1. Calculated Property Pattern

The implementation uses a **calculated property** rather than storing available quota directly:

```python
@property
def quota(self):
    return self.max_quota - self.quota_consumed
```

**Advantages:**
- Single source of truth (derived value, not stored)
- Eliminates synchronization bugs between multiple state variables
- Always guarantees consistent state

**Alternative Approach (Not Used):**
```python
# Store available quota directly
self.available_quota = 100
```

**Why Avoided:**
- Would require maintaining three variables in sync
- Higher risk of inconsistent state
- More complex state management logic

#### 2. Tri-Modal State Transition Logic

The setter implements three distinct operational modes:

**Scenario 1: Reset (amount == 0)**
```python
if amount == 0:
    self.quota_consumed = 0
    self.max_quota = 0
```
- Complete bucket reset
- Use case: Clearing rate limits, initialization

**Scenario 2: Initial Fill (delta < 0)**
```python
elif delta < 0:
    assert self.quota_consumed == 0
    self.max_quota = amount
```
- Setting initial capacity or increasing capacity
- Assertion prevents capacity changes mid-consumption
- Use case: First fill, capacity upgrades

**Scenario 3: Consumption (delta >= 0)**
```python
else:
    assert self.max_quota >= self.quota_consumed
    self.quota_consumed += delta
```
- Normal token consumption
- Tracks cumulative token usage
- Use case: Rate limiting operations

#### 3. Assertion-Based Contract Enforcement

The implementation uses assertions to enforce preconditions:

**Assertion 1:**
```python
assert self.quota_consumed == 0
```
- Ensures bucket is empty before capacity changes
- Prevents invalid state transitions
- Catches programming errors during development

**Assertion 2:**
```python
assert self.max_quota >= self.quota_consumed
```
- Validates bucket integrity
- Catches overconsumption bugs
- Ensures logical consistency of internal state

**⚠️ Production Consideration:**
Assertions are disabled with `python -O` flag. For production systems, consider replacing with explicit validation and exception handling.

---
## Part 7: Real-World Use Case - API Rate Limiting

In [ ]:
# Simulate API rate limiting: 100 requests per 60 seconds
api_bucket = NewBucket(60)
fill(api_bucket, 100)

print(f"API Rate Limiter initialized: {api_bucket.quota} requests available")
print()

# Simulate processing requests
def process_api_request(bucket, request_id):
    """Simulate processing an API request with rate limiting"""
    if bucket.quota > 0:
        # Consume one token
        bucket.quota = bucket.quota - 1
        print(f"✓ Request {request_id} processed. Remaining quota: {bucket.quota}")
        return True
    else:
        print(f"✗ Request {request_id} rejected. Rate limit exceeded!")
        return False

# Process some requests
for i in range(1, 6):
    process_api_request(api_bucket, i)

print(f"\nFinal state: {api_bucket}")

---
## Part 8: Potential Improvements

### 1. Production-Ready Validation

Replace assertions with explicit exception handling:

In [ ]:
class ProductionBucket(NewBucket):
    """Enhanced version with production-ready validation"""
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        """Setter with explicit exception handling instead of assertions"""
        delta = self.max_quota - amount
        
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Replace assertion with explicit validation
            if self.quota_consumed != 0:
                raise ValueError(
                    f"Cannot change capacity with consumed quota. "
                    f"Current consumption: {self.quota_consumed}"
                )
            self.max_quota = amount
        else:
            # Replace assertion with explicit validation
            if self.max_quota < self.quota_consumed:
                raise ValueError(
                    f"Invalid state: quota_consumed ({self.quota_consumed}) "
                    f"exceeds max_quota ({self.max_quota})"
                )
            self.quota_consumed += delta

# Test the production version
prod_bucket = ProductionBucket(60)
fill(prod_bucket, 50)
print(f"Production bucket: {prod_bucket}")

### 2. Auto-Refill Functionality

Add automatic time-based refill when period expires:

In [ ]:
class AutoRefillBucket(NewBucket):
    """Bucket with automatic time-based refill"""
    
    def _auto_refill(self):
        """Automatically refill tokens if period has elapsed"""
        now = datetime.now()
        elapsed = now - self.reset_time
        
        if elapsed >= self.period_delta:
            # Period has elapsed, reset consumption
            self.quota_consumed = 0
            self.reset_time = now
            print(f"[AUTO-REFILL] Bucket refilled to {self.max_quota} tokens")
    
    @property
    def quota(self):
        """Check for auto-refill before returning quota"""
        self._auto_refill()
        return self.max_quota - self.quota_consumed

# Demonstrate auto-refill (requires time delay to see effect)
auto_bucket = AutoRefillBucket(60)
fill(auto_bucket, 100)
print(f"Auto-refill bucket created: {auto_bucket}")

### 3. Thread-Safe Implementation

Add locking for concurrent access:

In [ ]:
from threading import Lock

class ThreadSafeBucket(NewBucket):
    """Thread-safe bucket implementation"""
    
    def __init__(self, period):
        super().__init__(period)
        self._lock = Lock()
    
    @property
    def quota(self):
        with self._lock:
            return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        with self._lock:
            delta = self.max_quota - amount
            
            if amount == 0:
                self.quota_consumed = 0
                self.max_quota = 0
            elif delta < 0:
                assert self.quota_consumed == 0
                self.max_quota = amount
            else:
                assert self.max_quota >= self.quota_consumed
                self.quota_consumed += delta

# Create thread-safe bucket
safe_bucket = ThreadSafeBucket(60)
fill(safe_bucket, 100)
print(f"Thread-safe bucket: {safe_bucket}")

---
## Part 9: Summary

### Key Takeaways

**Strengths of the Implementation:**
1. **Calculated Property Pattern** - Eliminates state synchronization issues
2. **Explicit State Transitions** - Clear tri-modal logic for different operations
3. **Defensive Programming** - Assertions catch logic errors during development
4. **Clean Interface** - Property decorator provides intuitive API

**Areas for Enhancement:**
1. **Missing Refill Logic** - No automatic token restoration over time
2. **Assertion Fragility** - Validation disappears in optimized deployments
3. **No Concurrency Control** - Not safe for multi-threaded environments
4. **Incomplete API** - Missing convenience methods for consumption

**Recommended Use Cases:**
- API rate limiting
- Bandwidth throttling
- Request quotas
- Resource allocation control
- Traffic shaping algorithms